In [1]:
import cv2
import mediapipe as mp
import time
import numpy as np
import HandTrackingModule as htm
import math

##############################
wCam, hCam = 640, 480
##############################

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, wCam)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, hCam)
pTime = 0

detector = htm.handDetector(detectionCon= 0.7)

# https://github.com/AndreMiras/pycaw
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()
minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400
volPer = 0

while 1:
    ret, img = cap.read()
    if ret == False:
        break

    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw= False)
    if len(lmList) != 0:
        x1, y1 = lmList[4][1:]
        x2, y2 = lmList[8][1:]
        cx, cy = (x1 + x2)//2, (y1 + y2)//2


        cv2.circle(img, (x1, y1), 10, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 10, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (cx, cy), 10, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1,y1), (x2, y2), (255, 0, 255), 3)

        length = math.hypot(x2 - x1, y2 - y1)
        # hand range = 50 - 100
        # Volume Range -65 - 0
        vol = np.interp(length, [50, 300], [minVol, maxVol])
        volBar = np.interp(length, [50, 300], [400, 150])
        volPer = np.interp(length, [50, 300], [0, 100])
        volume.SetMasterVolumeLevel(vol, None)

        if length < 50:
            cv2.line(img, (x1,y1), (x2, y2), (0, 255, 0), 3)

        cv2.rectangle(img, (50, 150), (85, 400), (250, 0, 0), 3)
        cv2.rectangle(img, (50, int(volBar)), (85, 400), (250, 0, 0), cv2.FILLED)
        cv2.putText(img, f"{int(volPer)} %",(40, 450),
                cv2.FONT_HERSHEY_COMPLEX, 1, (250, 0, 0), 3)

    cTime = time.time()
    fps = 1/(cTime - pTime)
    pTime = cTime

    cv2.putText(img, f"FPS: {int(fps)}",(40, 50),
        
                cv2.FONT_HERSHEY_COMPLEX, 1, (250, 0, 0), 3)

    cv2.imshow("img", img)
    if cv2.waitKey(10) == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()